# compare 2 methods of calling - concat vs individual

In [ ]:
import sys,os,shutil,subprocess
import random
import glob, time, platform
import tempfile
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import seaborn as sns
from Bio import SeqIO, AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from gzip import open as gzopen
sys.path.append('snipgenie')
from snipgenie import tools, aligners, app, tools

In [ ]:
reload(app)
ref = app.sarscov2_genome
args = {'threads':12, 'outdir': 'simdata/sim_results', 'labelsep':'.',
        'input':'simdata/sim_fastq/',        
        'reference': ref,
        'overwrite':False,
        'filters':'', 
        'custom_filters': False, 'get_stats':False}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

In [207]:
samples = pd.read_csv('simdata/sim_results/samples.csv')

In [116]:
def compare_results(c1, c2, sample=None):
    """
    Compare two runs of snipgenie.
    """

    x = c1[~c1.pos.isin(c2.pos)]
    y = c2[~c2.pos.isin(c1.pos)]
    print ('%s/%s sites not in second:' %(len(x),len(c1)))
    print (x)
    print ('-------------------------')
    print ('%s/%s sites not in first:' %(len(y),len(c2)))
    print (y)
    return x,y

In [106]:
import multiprocessing as mp

def worker(args):
    mpileup(args[0], args[1])
        
def mpileup_multiprocess(bam_files, ref, outpath, threads=4, callback=None):
    """Run mpileup in parallel over multiple files and make separate bcfs.
    Assumes alignment to a bacterial reference with a single chromosome."""

    bcftoolscmd = tools.get_cmd('bcftools')
    #size = len(bam_files) 
    #pool = mp.Pool(threads)
    outfiles = []
    st = time.time()
    bcfpath = os.path.join(outpath,'bcf')
    if not os.path.exists(bcfpath):
        os.mkdir(bcfpath)
    for bam_file in bam_files:
        name = os.path.splitext(os.path.basename(bam_file))[0]
        out = '{o}/{f}.bcf'.format(o=bcfpath,f=name) 
        outfiles.append(out)
   
    data = list(zip(bam_files,outfiles))
    #print (data)
   
    p = mp.Pool(threads)
    p.map_async(worker, data)
    p.close()
    p.join()
    
    t=time.time()-st
    print ('took %s seconds' %str(round(t,3)))
    rawbcf = os.path.join(outpath,'raw.bcf')
    bcf_files = ' '.join(outfiles)
    cmd = '{bc} merge --threads {t} -o {r} {b}'.format(b=bcf_files,r=rawbcf, bc=bcftoolscmd,t=threads)
    print (cmd)
    subprocess.check_output(cmd, shell=True)
    return rawbcf

def mpileup_files(bam_files, ref, outpath, threads=4):
    """mpileup bam files"""
    
    bcftoolscmd = tools.get_cmd('bcftools')
    outfiles = []
    st = time.time()
    bcfpath = os.path.join(outpath,'bcf')
    if not os.path.exists(bcfpath):
        os.mkdir(bcfpath)
    for bam_file in bam_files:
        name = os.path.splitext(os.path.basename(bam_file))[0]
        out = '{o}/{f}.bcf'.format(o=bcfpath,f=name) 
        outfiles.append(out)        
   
    data = list(zip(bam_files,outfiles)) 
    #print (data)
    for f,out in data:
        mpileup(f, out, ref, threads=threads)
        
    t=time.time()-st
    print ('took %s seconds' %str(round(t,3)))
    rawbcf = os.path.join(outpath,'raw.bcf')
    bcf_files = ' '.join(outfiles)
    #print (bcf_files)
    cmd = '{bc} merge --threads {t} -0 -o {o} {b}'.format(b=bcf_files,o=rawbcf, bc=bcftoolscmd,t=threads)
    print (cmd)
    #subprocess.check_output(cmd, shell=True)
    #cmd = '{bc} norm -f {r} -O b -o {o} tmp.bcf'.format(o=rawbcf, bc=bcftoolscmd, r=ref) 
    subprocess.check_output(cmd, shell=True)    
    return rawbcf

def mpileup(bam_file, out, ref, threads=4, overwrite=False):
    """Run bcftools for single file."""

    bcftoolscmd = tools.get_cmd('bcftools')
    if overwrite==False and os.path.exists(out):
        return
    cmd = '{bc} mpileup -a {a} -O b --max-depth 500 --min-MQ 60 -f {r} -o {o} {b}'\
            .format(r=ref, b=bam_file, o=out, bc=bcftoolscmd, a=app.annotatestr, t=threads)
    subprocess.check_output(cmd, shell=True)  
    print (cmd)
    cmd = 'bcftools index {o}'.format(o=out)
    subprocess.check_output(cmd, shell=True)
    return
   

In [225]:
annotatestr = '"AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR"'

def mpileup_parallel(bam_file, ref, rawbcf, threads=4, callback=None, tempdir=None):
    """Run mpileup in over multiple regions with GNU parallel on linux or rush on Windows
      Separate bcf files are then joined together.
      Assumes alignment to a bacterial reference with a single chromosome.
    """

    if tempdir == None:
        tempdir = tempfile.tempdir
    #bam_files = ' '.join(bam_files)
    #rawbcf = os.path.join(outpath,'raw.bcf')
    chr = tools.get_chrom(ref)
    length = tools.get_fasta_length(ref)
    x = np.linspace(1,length,threads+1,dtype=int)
    print (x)

    #split genome into blocks
    blocks=[]
    for i in range(len(x)):
        if i < len(x)-1:
            blocks.append((x[i],x[i+1]-1))

    #get temp outfile names
    outfiles = []
    regions = []
    for start,end in blocks:
        region = '"{c}":{s}-{e}'.format(c=chr,s=start,e=end)
        regions.append(region)
        out = os.path.join(tempdir,'{s}-{e}.bcf'.format(s=start,e=end))
        outfiles.append(out)

    regstr = ' '.join(regions)
    #print (regstr)
    filesstr = ' '.join(outfiles)
    bcftoolscmd = tools.get_cmd('bcftools')

    if platform.system() == 'Windows':
        rushcmd = tools.get_cmd('rush')
        cmd = 'echo {reg} | {rc} -D " " "{bc} mpileup -r {{}} -f {r} -a {a} --max-depth 500 --min-MQ 60 {b} -o {p}/{{@[^:]*$}}.bcf"'\
                .format(rc=rushcmd,bc=bcftoolscmd,reg=regstr,r=ref,b=bam_file,a=annotatestr,p=tempdir)
    else:
        cmd = 'parallel bcftools mpileup -r {{1}} -a {a} -O b --max-depth 500 --min-MQ 60 -o {{2}} -f {r} {b} ::: {reg} :::+ {o}'\
                .format(r=ref, reg=regstr, b=bam_file, o=filesstr, a=annotatestr)
    print (cmd)
    #if callback != None:
    #    callback(cmd)
    subprocess.check_output(cmd, shell=True)
    #concat the separate files
    cmd = '{bc} concat {i} --threads {t} -O b -o {o}'.format(bc=bcftoolscmd,i=' '.join(outfiles),o=rawbcf,t=threads)
    #print (cmd)
    subprocess.check_output(cmd, shell=True)
    #remove temp files
    for f in outfiles:
        os.remove(f)
    return rawbcf

In [227]:
reload(tools)

def run_file(bam_file, ref, outpath, filters=None, mask=None,
             overwrite=False, threads=4):
    """
    Run mpileup/snp calling/filtering for one file.
    """
    
    if not os.path.exists(outpath):
        os.makedirs(outpath, exist_ok=True)
    if filters == None:
        filters = app.default_filter        
    bcftoolscmd = tools.get_cmd('bcftools')
    rawbcf = os.path.join(outpath,'raw.bcf')
    
    if not os.path.exists(rawbcf) or overwrite == True:
        mpileup_parallel(bam_file, ref, rawbcf, threads=threads)
        #mpileup(bam_file, rawbcf, ref, threads=threads, overwrite=overwrite)
    
    #call
    vcfout = os.path.join(outpath,'calls.vcf')
    #assumes other files are there and returns - probably not ideal
    if os.path.exists(vcfout) and overwrite == False:
        return
    tools.bcftools_call(rawbcf, vcfout)#, show_cmd=True)
    #filter calls
    if filters != None:
        filtered = os.path.join(outpath,'filtered.bcf')
        cmd = '{bc} filter -i "{f}" -o {o} -O b {i}'.format(bc=bcftoolscmd,i=vcfout,o=filtered,f=filters)
        #print (cmd)
        tmp = subprocess.check_output(cmd,shell=True)
    else:
        filtered = vcfout
    #print ('splitting snps and indels..')
    snpsout = os.path.join(outpath,'snps.bcf')
    cmd = '{bc} view -v snps -o {o} -O b {i}'.format(bc=bcftoolscmd,o=snpsout,i=filtered)
    #print (cmd)
    subprocess.check_output(cmd,shell=True)
    cmd = 'bcftools index {o}'.format(o=snpsout)
    subprocess.check_output(cmd, shell=True)
    
    #also get indels only to separate file
    indelsout = os.path.join(outpath,'indels.bcf')
    #cmd = '{bc} call -V snps --ploidy 1 -m -v -o {o} {raw}'.format(bc=bcftoolscmd,o=indelsout,raw=rawbcf)
    cmd = '{bc} view -v indels -o {o} -O z {i}'.format(bc=bcftoolscmd,o=indelsout,i=filtered)
    #print (cmd)
    subprocess.check_output(cmd,shell=True)

    #apply mask if required
    if mask != None:
        mask_filter(snpsout, mask, outdir=outpath, overwrite=True)
        mask_filter(indelsout, mask, outdir=outpath, overwrite=True)  
    
    return snpsout

path = 'simdata/sim_results2/variant_calling/S1'
bam_file = 'simdata/sim_results/mapped/S1.bam'
run_file(bam_file, app.sarscov2_genome, path, overwrite=True)


[    1  7476 14952 22427 29903]
parallel bcftools mpileup -r {1} -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" -O b --max-depth 500 --min-MQ 60 -o {2} -f /home/farrell/.config/snipgenie/genome/Sars-Cov-2.fa simdata/sim_results/mapped/S1.bam ::: "NC_045512.2":1-7475 "NC_045512.2":7476-14951 "NC_045512.2":14952-22426 "NC_045512.2":22427-29902 :::+ /tmp/1-7475.bcf /tmp/7476-14951.bcf /tmp/14952-22426.bcf /tmp/22427-29902.bcf
bcftools concat /tmp/1-7475.bcf /tmp/7476-14951.bcf /tmp/14952-22426.bcf /tmp/22427-29902.bcf --threads 4 -O b -o simdata/sim_results2/variant_calling/S1/raw.bcf


[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
Checking the headers and starting positions of 4 files
Concatenating /tmp/1-7475.bcf	0.004075 seconds
Concatenating /tmp/7476-14951.bcf	0.001616 seconds
Concatenating /tmp/14952-22426.bcf	0.002544 seconds
Concatenating /tmp/22427-29902.bcf	0.002100 seconds


'simdata/sim_results2/variant_calling/S1/snps.bcf'

In [ ]:
def worker(args):
    run_file(args[0], args[1], args[2], args[3])  
        
def variant_calling_multiprocess(samples, ref, outpath, filters=None, custom_filters=False, mask=None,
                        overwrite=False, threads=4):
  
    bcftoolscmd = tools.get_cmd('bcftools')
    vcfoutpath = os.path.join(outpath, 'variant_calling')
    os.makedirs(vcfoutdir, exist_ok=True)    
    if samples is not None:
        #write out new samples.txt file for reheader step
        app.write_samples(samples[['sample']], outpath)
        sample_file = os.path.join(outpath,'samples.txt') 
        
    outpaths = []
    #outfiles = []
    for i,r in samples.iterrows():    
        name = r['sample']
        path = os.path.join(vcfoutpath, name)
        #snpfile = run_file(r.bam_file, ref, path, filters=filters, mask=mask)
        
        #snpfile = os.path.join(vcfoutpath, name, 'snps.bcf')
        outfiles.append(snpfile)
    
    data = list(zip(bam_files,outpaths,mask))
    print (data)
   
    #p = mp.Pool(threads)
    #p.map_async(worker, data)
    #p.close()
    #p.join()

    

outdir='simdata/sim_results_parallel/'
variant_calling_multiprocess(samples, app.sarscov2_genome, outdir, filters=None)

In [216]:
def new_variant_calling(samples, ref, outpath, filters=None, custom_filters=False, mask=None,
                        overwrite=False, threads=4):
    """
    Run variant calling for multiple samples
    """
    
    bcftoolscmd = tools.get_cmd('bcftools')
    vcfoutpath = os.path.join(outpath, 'variant_calling')
    os.makedirs(vcfoutdir, exist_ok=True)    
    if samples is not None:
        #write out new samples.txt file for reheader step
        app.write_samples(samples[['sample']], outpath)
        sample_file = os.path.join(outpath,'samples.txt')    
    outfiles = []
    for i,r in samples.iterrows():    
        name = r['sample']
        print (name)
        path = os.path.join(vcfoutpath, name)
        snpfile = run_file(r.bam_file, ref, path, filters=filters, mask=mask, threads=threads)
        outfiles.append(snpfile)

    merged = os.path.join(outpath, 'merged.vcf.gz')
    bcf_files = ' '.join(outfiles)
    #print (bcf_files)
    cmd = '{bc} merge --threads {t} -0 -O z -o {o} {f}'.format(f=bcf_files,o=merged, bc=bcftoolscmd,t=threads)
    print (cmd)
    subprocess.check_output(cmd, shell=True)  
    app.relabel_vcfheader(merged, sample_file)
    #custom filters
    if custom_filters == True:
        app.site_proximity_filter(merged, outdir=outpath, overwrite=True)
    return

outdir='simdata/sim_results2/'
new_variant_calling(samples, app.sarscov2_genome, outdir, filters=None)
app.run_vcf('simdata/sim_results2/merged.vcf.gz', outdir, threads=8)

S1
S10
S11
S2
S3
S4
S5
S6
S7
S8
S9
bcftools merge --threads 4 -0 -O z -o simdata/sim_results2/merged.vcf.gz simdata/sim_results2/variant_calling/S1/snps.bcf simdata/sim_results2/variant_calling/S10/snps.bcf simdata/sim_results2/variant_calling/S11/snps.bcf simdata/sim_results2/variant_calling/S2/snps.bcf simdata/sim_results2/variant_calling/S3/snps.bcf simdata/sim_results2/variant_calling/S4/snps.bcf simdata/sim_results2/variant_calling/S5/snps.bcf simdata/sim_results2/variant_calling/S6/snps.bcf simdata/sim_results2/variant_calling/S7/snps.bcf simdata/sim_results2/variant_calling/S8/snps.bcf simdata/sim_results2/variant_calling/S9/snps.bcf
bcftools reheader --samples simdata/sim_results2/samples.txt -o /tmp/calls.vcf simdata/sim_results2/merged.vcf.gz
getting core alignment..
uninformative_sites False
found 172 sites for core snps
0 sites with at least one missing sample
0 uninformative sites
computing snp distance matrix..
raxmlHPC-PTHREADS -f a -N 100 -T 8 -m GTRCAT -V -p 86013816 -

## SNIPPY test

In [ ]:
outdir='simdata/snippy_results'
ref=app.sarscov2_genome
for i, r in samples.iterrows():    
    out = os.path.join(outdir, r['sample'])
    cmd = '/local/snippy/bin/snippy --cpus 8 --outdir {o} --ref {r} --R1 {f1} --R2 {f2}'.format(
            o=out,r=ref,f1=r.filename1,f2=r.filename2)
    print (cmd)
    subprocess.check_output(cmd, shell=True)

In [ ]:
cmd = "/local/snippy/bin/snippy-core --ref 'simdata/snippy_results/S1/ref.fa' S1 S10 S11 S2 S3 S4 S5 S6 S7 S8 S9"
subprocess.check_output(cmd, shell=True)

In [211]:
c1=pd.read_csv('simdata/sim_results/core.txt',sep=' ')
c2=pd.read_csv('simdata/sim_results2/core.txt',sep=' ')
c3=pd.read_csv('simdata/snippy_results/core.tab',sep='\t')
c3=c3.rename(columns={'POS':'pos'})
x,y=compare_results(c1,c2)


4/176 sites not in second:
       pos ref S1 S10 S11 S2 S3 S4 S5 S6 S7 S8 S9
25    3584   G  G   G   G  G  G  A  G  G  G  G  G
30    4588   A  A   A   G  A  A  A  A  A  A  A  A
31    4626   C  C   C   T  C  C  C  C  C  C  C  C
166  28546   C  C   C   C  C  C  C  C  C  C  C  T
-------------------------
0/172 sites not in first:
Empty DataFrame
Columns: [pos, ref, S1, S10, S11, S2, S3, S4, S5, S6, S7, S8, S9]
Index: []


In [ ]:
d1 = pd.read_csv('simdata/sim_results/snpdist.csv',index_col=0)
d2 = pd.read_csv('simdata/sim_results2/snpdist.csv',index_col=0)
print (d1)
print (d2)

## test with wicklow data

In [206]:
wsamples = pd.read_csv('/storage/btbgenie/wicklow_results/samples.csv')
bam_files = list(samples.bam_file.unique())
ref = app.mbovis_genome


In [ ]:
reload(app)


In [ ]:
v1 = tools.vcf_to_dataframe(os.path.join('wicklow_results','snps.vcf.gz'))
v1
print (tools.bcftools_query('wicklow_results/raw.bcf',[205557]))
print (tools.bcftools_query('wicklow_temp/raw.bcf',[205557]))

In [ ]:
compare_results('/storage/btbgenie/wicklow_results','wicklow_temp')
compare_results('wicklow_temp','/storage/btbgenie/wicklow_results')

In [ ]:
v1 = vcf_to_dataframe('result.vcf')
print (v1.var_type.value_counts())

In [ ]:
v2 = vcf_to_dataframe('mapped/raw.bcf')
print (v2.var_type.value_counts())

## run Ireland samples with new method

In [ ]:
outdir='/storage/btbgenie/all_ireland_results_new'
iresamples = pd.read_csv('/storage/btbgenie/all_ireland_results/samples.csv')
new_variant_calling(iresamples, app.mbovis_genome, outdir, filters=None, threads=12)
app.run_vcf(f'{outdir}/merged.vcf.gz', outdir, threads=12)

	0.254752 seconds
Concatenating /tmp/3987412-4349903.bcf	0.277171 seconds


TB19-011147
[      1  362492  724984 1087476 1449968 1812460 2174952 2537444 2899936
 3262428 3624920 3987412 4349904]
parallel bcftools mpileup -r {1} -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" -O b --max-depth 500 --min-MQ 60 -o {2} -f /home/farrell/.config/snipgenie/genome/Mbovis_AF212297.fa /storage/btbgenie/all_ireland_results/mapped/TB19-011147.bam ::: "LT708304.1":1-362491 "LT708304.1":362492-724983 "LT708304.1":724984-1087475 "LT708304.1":1087476-1449967 "LT708304.1":1449968-1812459 "LT708304.1":1812460-2174951 "LT708304.1":2174952-2537443 "LT708304.1":2537444-2899935 "LT708304.1":2899936-3262427 "LT708304.1":3262428-3624919 "LT708304.1":3624920-3987411 "LT708304.1":3987412-4349903 :::+ /tmp/1-362491.bcf /tmp/362492-724983.bcf /tmp/724984-1087475.bcf /tmp/1087476-1449967.bcf /tmp/1449968-1812459.bcf /tmp/1812460-2174951.bcf /tmp/2174952-2537443.bcf /tmp/2537444-2899935.bcf /tmp/2899936-3262427.bcf /tmp/3262428-3624919.bcf /tmp/3624920-3987411.bcf /tmp/3987412-4349903.bcf


[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500


bcftools concat /tmp/1-362491.bcf /tmp/362492-724983.bcf /tmp/724984-1087475.bcf /tmp/1087476-1449967.bcf /tmp/1449968-1812459.bcf /tmp/1812460-2174951.bcf /tmp/2174952-2537443.bcf /tmp/2537444-2899935.bcf /tmp/2899936-3262427.bcf /tmp/3262428-3624919.bcf /tmp/3624920-3987411.bcf /tmp/3987412-4349903.bcf --threads 12 -O b -o /storage/btbgenie/all_ireland_results_new/variant_calling/TB19-011147/raw.bcf


	0.175770 seconds
Concatenating /tmp/362492-724983.bcf	0.306782 seconds
Concatenating /tmp/724984-1087475.bcf	0.242226 seconds
Concatenating /tmp/1087476-1449967.bcf	0.240516 seconds
Concatenating /tmp/1449968-1812459.bcf	0.229551 seconds
Concatenating /tmp/1812460-2174951.bcf	0.220389 seconds
Concatenating /tmp/2174952-2537443.bcf	0.223001 seconds
Concatenating /tmp/2537444-2899935.bcf	0.228172 seconds
Concatenating /tmp/2899936-3262427.bcf	0.234088 seconds
Concatenating /tmp/3262428-3624919.bcf	0.249381 seconds
Concatenating /tmp/3624920-3987411.bcf	0.218548 seconds
Concatenating /tmp/3987412-4349903.bcf	0.243348 seconds


...

[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500


bcftools concat /tmp/1-362491.bcf /tmp/362492-724983.bcf /tmp/724984-1087475.bcf /tmp/1087476-1449967.bcf /tmp/1449968-1812459.bcf /tmp/1812460-2174951.bcf /tmp/2174952-2537443.bcf /tmp/2537444-2899935.bcf /tmp/2899936-3262427.bcf /tmp/3262428-3624919.bcf /tmp/3624920-3987411.bcf /tmp/3987412-4349903.bcf --threads 12 -O b -o /storage/btbgenie/all_ireland_results_new/variant_calling/TB19-010884/raw.bcf


	0.259199 seconds
Concatenating /tmp/362492-724983.bcf	0.258796 seconds
Concatenating /tmp/724984-1087475.bcf	0.243366 seconds
Concatenating /tmp/1087476-1449967.bcf	0.264793 seconds
Concatenating /tmp/1449968-1812459.bcf	0.252129 seconds
Concatenating /tmp/1812460-2174951.bcf	0.276069 seconds
Concatenating /tmp/2174952-2537443.bcf	0.266437 seconds
Concatenating /tmp/2537444-2899935.bcf	0.266962 seconds
Concatenating /tmp/2899936-3262427.bcf	0.274912 seconds
Concatenating /tmp/3262428-3624919.bcf	0.256782 seconds
Concatenating /tmp/3624920-3987411.bcf	0.269302 seconds
Concatenating /tmp/3987412-4349903.bcf	0.275111 seconds


TB19-010868
[      1  362492  724984 1087476 1449968 1812460 2174952 2537444 2899936
 3262428 3624920 3987412 4349904]
parallel bcftools mpileup -r {1} -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" -O b --max-depth 500 --min-MQ 60 -o {2} -f /home/farrell/.config/snipgenie/genome/Mbovis_AF212297.fa /storage/btbgenie/all_ireland_results/mapped/TB19-010868.bam ::: "LT708304.1":1-362491 "LT708304.1":362492-724983 "LT708304.1":724984-1087475 "LT708304.1":1087476-1449967 "LT708304.1":1449968-1812459 "LT708304.1":1812460-2174951 "LT708304.1":2174952-2537443 "LT708304.1":2537444-2899935 "LT708304.1":2899936-3262427 "LT708304.1":3262428-3624919 "LT708304.1":3624920-3987411 "LT708304.1":3987412-4349903 :::+ /tmp/1-362491.bcf /tmp/362492-724983.bcf /tmp/724984-1087475.bcf /tmp/1087476-1449967.bcf /tmp/1449968-1812459.bcf /tmp/1812460-2174951.bcf /tmp/2174952-2537443.bcf /tmp/2537444-2899935.bcf /tmp/2899936-3262427.bcf /tmp/3262428-3624919.bcf /tmp/3624920-3987411.bcf /tmp/3987412-4349903.bcf


[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 500
